In [2]:
import tensorflow as tf
print(tf.__version__)


2.18.0


In [3]:
import os
import pandas as pd
import tensorflow as ts
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical

from keras import backend as K
import pickle

In [4]:

os.chdir("C:/Users/Kanav/Documents/urlclassifier/FinalDataset")

In [5]:
data = pd.read_csv("feature.csv")

In [6]:
data.head()

,Unnamed: 0,File,bodyLength,bscr,dse,dsr,entropy,hasHttp,hasHttps,has_ip,...,numParams,numTitles,num_%20,num_@,sbr,scriptLength,specialChars,sscr,urlIsLive,urlLength
0,0,spam_dataset,31,0.064516,0,0,-4.708307,True,False,0,...,0,0,0,0,0.000000,0,2,0.000000,False,81
1,1,spam_dataset,64790,0.306050,0,0,-4.576882,True,False,0,...,0,19,0,0,0.923800,59853,19829,3.018458,False,58
2,2,spam_dataset,757,0.195509,0,0,-4.205536,True,False,0,...,0,1,0,0,0.000000,0,148,0.000000,False,46
3,3,spam_dataset,266310,0.230915,0,0,-4.550931,True,False,0,...,0,648,0,0,0.312647,83261,61495,1.353947,False,52
4,4,spam_dataset,40,0.200000,0,0,-4.689336,True,False,0,...,2,1,0,0,0.000000,0,8,0.000000,False,83


In [7]:
set(data['File'])

{'Benign_list_big_final',
 'Defacement',
 'Malware_dataset',
 'phishing_dataset',
 'spam_dataset'}

In [8]:
data.replace("Benign_list_big_final","Benign",inplace=True)
data.replace("Malware_dataset","Malware",inplace=True)
data.replace("phishing_dataset","Phishing",inplace=True)
data.replace("spam_dataset","Spam",inplace=True)

In [9]:
data.drop(columns='Unnamed: 0',inplace=True)
data.replace(True,1,inplace = True)
data.replace(False,0,inplace = True)
y = data["File"]

C:\Users\Kanav\AppData\Local\Temp\ipykernel_15328\387194231.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace(False,0,inplace = True)


In [10]:
data = data.drop(columns = "File")

In [11]:
encoder = LabelEncoder()
encoder.fit(y)
Y = encoder.transform(y)

In [12]:
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(data)
X = pd.DataFrame(X)

In [13]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

input_dim = len(data.columns)
model = Sequential()
model.add(Dense(256, input_dim = input_dim , activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )

c:\Users\Kanav\Documents\urlclassifier\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [15]:
model.fit(X_train, to_categorical(y_train), epochs=50, validation_split=0.3, batch_size=128)


Epoch 1/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.3582 - loss: 1.4405 - val_accuracy: 0.5834 - val_loss: 1.0231
Epoch 2/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6082 - loss: 0.9724 - val_accuracy: 0.6518 - val_loss: 0.8733
Epoch 3/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6567 - loss: 0.8641 - val_accuracy: 0.6742 - val_loss: 0.8217
Epoch 4/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6786 - loss: 0.8132 - val_accuracy: 0.6951 - val_loss: 0.7572
Epoch 5/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6961 - loss: 0.7665 - val_accuracy: 0.7165 - val_loss: 0.7085
Epoch 6/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7185 - loss: 0.7087 - val_accuracy: 0.7267 - val_loss: 0.6929
Epoch 7/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7323 - loss: 0.6758 - val_accuracy: 0.7161 - val_loss: 0.6951
Epoch 8/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7396 - loss: 0.6620 - val_accuracy: 0.

In [16]:
y_pred = model.predict(X_test)

427/427 ━━━━━━━━━━━━━━━━━━━━ 0s 733us/step


In [17]:
predicted = np.argmax(y_pred, axis=1)

In [18]:
from sklearn.metrics import classification_report
print(accuracy_score(y_test,predicted))

0.8519768209491675


In [19]:
target_names = ['Benign','Defacement','Malware','Phishing','Spam']
print(classification_report(y_test, predicted, target_names=target_names))

              precision    recall  f1-score   support

      Benign       0.93      0.95      0.94      2735
  Defacement       0.79      0.73      0.76      2499
     Malware       0.84      0.91      0.88      2834
    Phishing       0.82      0.73      0.77      2477
        Spam       0.86      0.90      0.88      3088

    accuracy                           0.85     13633
   macro avg       0.85      0.85      0.85     13633
weighted avg       0.85      0.85      0.85     13633



In [26]:
os.chdir("../")
os.chdir("models")
model.save("Model_v2.keras")
np.save('lblenc_v1.npy', encoder.classes_)
scalerfile = 'scaler_v1.sav'
pickle.dump(scaler, open(scalerfile, 'wb'))

In [27]:
import pathlib

In [28]:

model = ts.keras.models.load_model('Model_v2.keras')
converter = ts.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("tflite_model.tflite", "wb").write(tflite_model)

c:\Users\Kanav\Documents\urlclassifier\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


INFO:tensorflow:Assets written to: C:\Users\Kanav\AppData\Local\Temp\tmpd9fkej86\assets


INFO:tensorflow:Assets written to: C:\Users\Kanav\AppData\Local\Temp\tmpd9fkej86\assets


Saved artifact at 'C:\Users\Kanav\AppData\Local\Temp\tmpd9fkej86'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 21), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  2064219102224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2064153688400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2064153687248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2064153686480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2064153689552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2064153686096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2064153687824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2064153688592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2064153688976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2064153689360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2064153689168: 

201136

In [29]:
model = ts.keras.models.load_model('Model_v2.keras')
converter = ts.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [ts.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

c:\Users\Kanav\Documents\urlclassifier\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


INFO:tensorflow:Assets written to: C:\Users\Kanav\AppData\Local\Temp\tmp74e8bpq3\assets


INFO:tensorflow:Assets written to: C:\Users\Kanav\AppData\Local\Temp\tmp74e8bpq3\assets


Saved artifact at 'C:\Users\Kanav\AppData\Local\Temp\tmp74e8bpq3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 21), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  2064160290192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2064160289808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2064160290000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2064160288848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2064160288656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2064160288080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2064160287888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2064160287312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2064160287120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2064160286544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2064160286352: 

In [30]:
tflite_model_file = pathlib.Path('tflite_quant_model.tflite')
tflite_model_file.write_bytes(tflite_quant_model)

61848